### LSTM总览

于 GRU 一样，LSTM 也是在 RNN 的基础上，希望控制历史信息的保留程度而设计出来的

相比 GRU，LSTM 有更多的中间值

同时也多一个潜变量（需要使用历史值进行更新的变量），也就是记忆细胞状态

遗忘门（Forget Gate），决定保留多少旧的细胞状态：

$$
f_t = \sigma(W_f x_t + U_f h_{t-1} + b_f)
$$

输入门（Input Gate），控制当前输入能带入多少新信息：

$$
i_t = \sigma(W_i x_t + U_i h_{t-1} + b_i)
$$

输出门（Output Gate），决定输出多少当前状态：

$$
o_t = \sigma(W_o x_t + U_o h_{t-1} + b_o)
$$

候选细胞状态（Candidate Cell State），生成候选的新记忆：

$$
\tilde{c}_t = \tanh(W_c x_t + U_c h_{t-1} + b_c)
$$

更新细胞状态，组合旧记忆和新记忆：

$$
c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t
$$

最终隐藏状态（Hidden State），由当前细胞状态和输出门共同决定：

$$
h_t = o_t \odot \tanh(c_t)
$$

使用 LSTM 预测一个 token 的过程图解如下：

![](md-img/LSTM.jpg)

$$
其中 C 和 H 都是潜变量，且 H、F、I、O、C、\tilde{C} 的形状一致
$$

<br>

### 代码从零实现

In [ ]:
import torch

class LSTMModel:
    # 保存模型参数
    def __init__(self, vocab_size, hiden_size):
        self.vocab_size = vocab_size
        self.hiden_size = hiden_size

        self.w_hf = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xf = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_f = torch.zeros(hiden_size)
        self.w_hi = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xi = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_i = torch.zeros(hiden_size)
        self.w_ho = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xo = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_o = torch.zeros(hiden_size)
        self.w_hc = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xc = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_c = torch.zeros(hiden_size)
        self.w_hy = torch.randn(hiden_size, vocab_size) * 0.01
        self.b_y = torch.zeros(vocab_size)

        self.parameters = [self.w_hf, self.w_xf, self.b_f,
                           self.w_hi, self.w_xi, self.b_i,
                           self.w_ho, self.w_xo, self.b_o,
                           self.w_hc, self.w_xc, self.b_c,
                           self.w_hy, self.b_y]
        
        for param in self.parameters:
            param.requires_grad_(True)

    # 正向传播
    # 输入数据形状为 (time_step, batch_size, vocab_size)
    def forward(self, X):
        h = torch.zeros(X.shape[1], self.hiden_size)    # 初始化隐藏状态
        c = torch.zeros(X.shape[1], self.hiden_size)    # 初始化记忆细胞

        Y = []   # 用于保存所有的预测输出

        # 按找时间步长，往后推算每一个样本的潜变量（隐藏状态、记忆细胞）
        for x in X:
            f = torch.sigmoid(h @ self.w_hf + x @ self.w_xf + self.b_f)    # 遗忘门
            i = torch.sigmoid(h @ self.w_hi + x @ self.w_xi + self.b_i)    # 输入门
            o = torch.sigmoid(h @ self.w_ho + x @ self.w_xo + self.b_o)    # 输出门
            c_ = torch.tanh(h @ self.w_hc + x @ self.w_xc + self.b_c)   # 候选记忆细胞

            c = f * c + i * c_       # 更新记忆细胞
            h = o * torch.tanh(c)    # 更新隐藏状态

            # 使用隐藏状态获取最终输出
            output = h @ self.w_hy + self.b_y
            Y.append(output)
        
        # 此处返回的 Y 的形状为 (time_step * batch_size, vocab_size)，方便计算损失函数
        return torch.cat(Y, dim=0), (h, c)